# Road Following - Live demo

In this notebook, we will use model we trained to move jetBot smoothly on track. 

### Load Trained Model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebook's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` in this notebook's directory.

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [1]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

Next, load the trained weights from the ``best_steering_model_xy.pth`` file that you uploaded.

In [2]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Creating the Pre-Processing Function

We have now loaded our model, but there's a slight issue. The format that we trained our model doesn't exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [5]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()
image_widget_mask = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)
display(image_widget_mask)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'')

We'll also create our robot instance which we'll need to drive the motors.

In [6]:
from jetbot import Robot

robot = Robot()

Now, we will define sliders to control JetBot
> Note: We have initialize the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_slider): If you see JetBot is wobbling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

In [7]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Next, let's display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

In [8]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [9]:
# Define the color that needs to be recognized.

#Yellow #FFFF00 works
colorUpper = np.array([44, 255, 255])
colorLower = np.array([24, 100, 100])

# Red FF0000
#colorUpper = np.array([180, 255, 255])
#colorLower = np.array([160, 100, 100])

In [16]:
# Position tolerance
x_lower = 50*(224/300)
x_upper = 250*(224/300)
y_lower = 250*(224/300)
y_upper = 300*(224/300)
min_radius = 20*(224/300)

In [18]:
# Color recognition and tracking function.
def findColor(imageInput):
    result = False
    result_X = False
    result_Y = False
    
    # Convert video frames to HSV color space.
    hsv = cv2.cvtColor(imageInput, cv2.COLOR_BGR2HSV)
    
    # Create a mask for pixels that match the target color.
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    
    # Erode, this process will remove the relatively 
    # small area in the mask just selected, which can be understood as denoising.
    mask = cv2.erode(mask, None, iterations=2)
    
    # dilate, the corrosion process just now will cause the large area to become 
    # smaller and the small area to disappear. This step is to restore the large area to its previous size.
    mask = cv2.dilate(mask, None, iterations=2)
    image_widget_mask.value = bgr8_to_jpeg(mask)
    
    # Obtain the conformed area contour.
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None
    
    # If there is a matching area, start to control the movement of the steering gear to achieve color tracking.
    if len(cnts) > 0:
        # Draw text to show that the target has been found.
        imageInput = cv2.putText(imageInput,'Target Detected',(10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
        
        # Find the contour of the largest area.
        c = max(cnts, key=cv2.contourArea)
        
        # Get the location of the center point of this area and the radius of this area.
        ((box_x, box_y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        # X, Y are the center points of the area.
        X = int(box_x)
        Y = int(box_y)
        imageInput = cv2.putText(imageInput,f'X: {X}, Y: {Y}',(100,110), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
        
        # error_X, error_Y are the absolute value of the error 
        # between the center point of the area and the center point of the frame.
        error_Y = abs(150 - Y)
        error_X = abs(150 - X)
        
        # Draw the size and position of this area.
        cv2.rectangle(imageInput,(int(box_x-radius),int(box_y+radius)),(int(box_x+radius),int(box_y-radius)),(255,255,255),1)
        
        if radius > min_radius:
            imageInput = cv2.putText(imageInput,'Size ok!',(150,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            
            if Y > y_lower and Y < y_upper:
                result_Y = True
                imageInput = cv2.putText(imageInput,'Object in Y-Area!',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)

            if X > x_lower and X < x_upper:
                result_X = True
                # Camera looks left.
                imageInput = cv2.putText(imageInput,'Object in X-Area!',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)

    # If no area matching the target color is found, the camera stops rotating.
    else:
        imageInput = cv2.putText(imageInput,'Target Detecting',(10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
    
    result = result_X and result_Y
    image_widget.value = bgr8_to_jpeg(imageInput)
    
    return imageInput, result

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

In [31]:
import time

angle = 0.0
angle_last = 0.0

# Initialize a global variable to store the last detection time
last_detection_time = 0
detection_cooldown = 5  # Cooldown period in seconds

def execute(change):
    global angle, angle_last, last_detection_time
    image = change['new']
    
    # Step 1: Check if within cooldown period
    current_time = time.time()
    if current_time - last_detection_time > detection_cooldown:
        color_detected = False
        _, color_detected = findColor(image)
        if color_detected:
            # Update the last detection time to start the cooldown period
            last_detection_time = current_time
            
            # Make U-Turn
            robot.left_motor.value = 0.75
            robot.right_motor.value = -0.75
            time.sleep(3.2)
            robot.left_motor.value = 0
            robot.right_motor.value = 0
            
            # Return and do next cycle
            return
    
    
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

In [32]:
execute({'new': camera.value})

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [33]:
camera.unobserve_all()
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [34]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

Again, let's close the camera conneciton properly so that we can use the camera in other notebooks.

In [35]:
camera.stop()

### Conclusion
That's it for this live demo! Hopefully you had some fun seeing your JetBot moving smoothly on track following the road!!!

If your JetBot wasn't following road very well, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)